In [36]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

### Raw Data Loading

In [37]:
pagos_raw = pd.read_csv("../../data/raw/expensas_pagos.csv", 
                    sep=";", 
                    na_values = "NULL",
                    dtype = {
                        "id" : "object",
                        "expensa_id" :"object"
                    })

expensas_raw = pd.read_csv("../../data/raw/expensas.csv",
                        sep=";", 
                        na_values = "NULL",
                        dtype = {
                            "id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object"
                        })

consorcios_raw = pd.read_csv("../../data/raw/expensas_consorcios.csv",
                         sep = ";",
                         na_values = "NULL",
                         dtype = {
                            "id": "object",
                        })

unidades_raw = pd.read_csv("../../data/raw/expensas_unidades.csv",
                         sep = ";",
                         na_values = "NULL",
                         dtype = {
                            "id": "object",
                        })

metodos_pago_raw = pd.read_csv("../../data/raw/expensas_pagos_metodos.csv",
                         sep = ";",
                         na_values = "NULL")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data Filtering and Preprocessing

In [41]:
def remove_deleted(data):
    return data.loc[data.deleted == 0, :]

def fix_wrong_dates(expensas, consorcios):
    data = pd.merge(expensas,
                    consorcios[["id", "modoPago", "visibilidadGastos","tipo"]],
                    left_on="consorcio_id",
                    right_on="id",
                    suffixes = ("","_consorcio"))
    
    vencido_12_selection = ((data.fecha_vencimiento_1 == '0000-00-00') 
                            | (data.fecha_vencimiento_2 == '0000-00-00')
                           | (data.fecha_vencimiento_1.str.contains('-00'))
                           | (data.fecha_vencimiento_2.str.contains('-00'))) & (data.modoPago == 'mes_vencido') & (data.mes == 12)
    vencido_resto_selection = ((data.fecha_vencimiento_1 == '0000-00-00') | (data.fecha_vencimiento_2 == '0000-00-00') | (data.fecha_vencimiento_1.str.contains('-00'))
                           | (data.fecha_vencimiento_2.str.contains('-00'))) & (data.modoPago == 'mes_vencido') & (data.mes != 12)
    adelantado_selection = ((data.fecha_vencimiento_1 == '0000-00-00') | (data.fecha_vencimiento_2 == '0000-00-00')| (data.fecha_vencimiento_1.str.contains('-00'))
                           | (data.fecha_vencimiento_2.str.contains('-00'))) & (data.modoPago == 'mes_adelantado')
    

    #vencido_12_selection
    data.loc[vencido_12_selection, "fecha_vencimiento_1"] = (data.loc[vencido_12_selection, "anio"] + 1).map(str) + "-01-10"
    data.loc[vencido_12_selection, "fecha_vencimiento_2"] = (data.loc[vencido_12_selection, "anio"] + 1).map(str) + "-01-20"

    #vencido_resto_selection
    data.loc[vencido_resto_selection, "fecha_vencimiento_1"] = data.loc[vencido_resto_selection, "anio"].map(str) + "-" + (data.loc[vencido_resto_selection, "mes"] + 1).apply(lambda x: '{0:0>2}'.format(x)) + "-10"
    data.loc[vencido_resto_selection, "fecha_vencimiento_2"] = data.loc[vencido_resto_selection, "anio"].map(str) + "-" + (data.loc[vencido_resto_selection, "mes"] + 1).apply(lambda x: '{0:0>2}'.format(x)) + "-20"
    
    #adelantado_12_selection
    data.loc[adelantado_selection, "fecha_vencimiento_1"] = data.loc[adelantado_selection, "anio"].map(str) + "-" + data.loc[adelantado_selection, "mes"].apply(lambda x: '{0:0>2}'.format(x)) + "-10"
    data.loc[adelantado_selection, "fecha_vencimiento_2"] = data.loc[adelantado_selection, "anio"].map(str) + "-" + data.loc[adelantado_selection, "mes"].apply(lambda x: '{0:0>2}'.format(x)) + "-20"    
    
    return data

def filter_expenses_concepts(expensas, concept_list = [1,10, 599]):
    return expensas.loc[expensas.concepto_id.isin(concept_list),:]

def filter_expenses_debts(expensas):
    return expensas.loc[expensas.monto != expensas.monto_total,:]

def remove_zero_payments(pagos):
    return pagos.loc[pagos.monto_pago > 0,:]

def filter_negative_amounts(expensas):
    return expensas.loc[(expensas.monto > 0) & (expensas.monto_total > 0),:]

def filter_expenses(expensas_raw, consorcios_raw):
    exp = remove_deleted(expensas_raw)
    exp = fix_wrong_dates(exp, consorcios_raw)
    exp = filter_expenses_concepts(exp)
    exp = filter_expenses_debts(exp)
    exp = filter_negative_amounts(exp)
    exp.fecha_vencimiento_1 = pd.to_datetime(exp.fecha_vencimiento_1)
    exp.fecha_vencimiento_2 = pd.to_datetime(exp.fecha_vencimiento_2)
    exp.fecha_emision = pd.to_datetime(exp.fecha_emision)
    return exp

def filter_payments(pagos_raw, metodos_pago_raw):
    pagos = pd.merge(pagos_raw,
                 metodos_pago_raw,
                 left_on="metodo",
                 right_on="id",
                 suffixes = ("","_mp"))

    pagos = pagos[["fecha","monto","expensa_id","nombre"]]
    pagos.columns = ["fecha_pago", "monto_pago", "expensa_id", "metodo_pago"]
    pagos = remove_zero_payments(pagos)
    pagos.fecha_pago = pd.to_datetime(pagos.fecha_pago, errors="coerce")
    return pagos

In [42]:
expensas = filter_expenses(expensas_raw, consorcios_raw)
pagos = filter_payments(pagos_raw, metodos_pago_raw)
unidades = unidades_raw[["id","denomination","consorcio_id", "meters","type_id"]]

In [43]:
expensas.to_csv('../../data/processed/expensas.csv')
pagos.to_csv('../../data/processed/pagos.csv')
unidades.to_csv('../../data/processed/unidades.csv')